In [51]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import json
import argparse

import config
from GPT import GPT
from LLAMA import LLAMA
from StimulusModel import LMFeatures
from utils_stim import get_stim, get_story_wordseqs
from utils_resp import get_resp
from utils_ridge.ridge import ridge, bootstrap_ridge
from utils_ridge.stimulus_utils import TRFile, load_textgrids, load_simulated_trfiles
from utils_ridge.dsutils import make_word_ds
from utils_ridge.interpdata import lanczosinterp2D
from utils_ridge.util import make_delayed
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import utils_llama.activation as ana

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
model_dir = '/ossfs/workspace/nas/gzhch/data/models/Llama-2-7b-hf'
model = AutoModelForCausalLM.from_pretrained(
    model_dir, 
    device_map='auto',
    torch_dtype=torch.float16,
).eval()
tokenizer = AutoTokenizer.from_pretrained(model_dir)

2024-01-04 16:03:32,727 - accelerate.utils.modeling - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 2/2 [08:24<00:00, 252.42s/it]


In [4]:
class ARGS:
    def __init__(self):
        self.subject = 'S1'
        self.gpt = 'perceived'
        self.sessions = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 18, 20]

args = ARGS()

In [5]:
# training stories
stories = []
with open(os.path.join(config.DATA_TRAIN_DIR, "sess_to_story.json"), "r") as f:
    sess_to_story = json.load(f) 
for sess in args.sessions:
    stories.extend(sess_to_story[str(sess)])

In [ ]:
word_seqs = get_story_wordseqs(stories)

In [54]:
llama = LLAMA(model, tokenizer, '/ossfs/workspace/act_cache_ffn_gate')

word_vecs = {}
for story in stories:
    words = word_seqs[story].data
    print(story, len(word_seqs[story].data))
    layer = 20
    act_name = 'ffn_gate'
    embs = llama.get_llm_act(story, words, config.GPT_WORDS, act_name, layer)
    word_vecs[story] = embs

alternateithicatom 2174
souls 1868
avatar 1469
legacy 1893
odetostepfather 2675
undertheinfluence 1641
howtodraw 1964
myfirstdaywiththeyankees 2786
naked 3218
life 2209
stagefright 2067
tildeath 2297
sloth 2403
exorcism 2949
onlyonewaytofindout 1889
penpal 1592
goingthelibertyway 2471
kiksuya 1699
thepostmanalwayscalls 2220
backsideofthestorm 1964
sweetaspie 697
thetiniestbouquet 964
lifeanddeathontheoregontrail 2389
thefreedomridersandme 1606
thumbsup 3083
becomingindian 2619
waitingtogo 1587
singlewomanseekingmanwich 1486
whenmothersbullyback 1488
superheroesjustforeachother 1440
gpsformylostidentity 1650
catfishingstrangerstofindmyself 1522
christmas1940 1261
stumblinginthedark 2681
forgettingfear 1240
bluehope 1941
lifereimagined 1800
ifthishaircouldtalk 1854
againstthewind 838
goldiethegoldfish 1680
quietfire 1905
cocoonoflove 1984
comingofageondeathrow 2212
leavingbaghdad 1976
mybackseatviewofagreatromance 1794
reachingoutbetweenthebars 1490
afatherscover 1182
wildwomenanddancing

In [37]:
word_mat = np.vstack([word_vecs[story] for story in stories])
word_mean, word_std = word_mat.mean(0), word_mat.std(0)

/opt/conda/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/opt/conda/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


In [42]:
ds_vecs = {story : lanczosinterp2D(word_vecs[story], word_seqs[story].data_times, word_seqs[story].tr_times) 
           for story in stories}

In [46]:
ds_mat = np.vstack([ds_vecs[story][5+config.TRIM:-config.TRIM] for story in stories])
r_mean, r_std = ds_mat.mean(0), ds_mat.std(0)
r_std[r_std == 0] = 1
ds_mat = np.nan_to_num(np.dot((ds_mat - r_mean), np.linalg.inv(np.diag(r_std))))
del_mat = make_delayed(ds_mat, config.STIM_DELAYS)

In [55]:
ds_mat.shape

(27449, 4096)

In [50]:
del_mat.shape

(27449, 16384)